<a href="https://colab.research.google.com/github/udlbook/udlbook/blob/main/Notebooks/Chap07/7_2_Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook 7.2: Backpropagation**

This notebook runs the backpropagation algorithm on a deep neural network as described in section 7.4 of the book.

Work through the cells below, running each cell in turn. In various places you will see the words "TO DO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

\



---



### 1. 신경망 구현

- 정의된 신경망 구조에 따라 초기 파라피터 값을 랜덤으로 설정

In [ ]:
# 랜덤 시드 생성, 난수 추출 시 고정된 값들이 출력됨
np.random.seed(0)  # 0이상의 정수

K = 5  # 은닉층
D = 6  # 은닉뉴런
D_i = 1  # 입력층
D_o = 1  # 출력층

# 파라미터 초기화
all_weights = [None] * (K+1)  # [None, None, None, None, None, None] - 길이가 6인 리스트
all_biases = [None] * (K+1)  # [None, None, None, None, None, None]
# 리스트의 길이는 신경망의 레이어 수(K)에 따라 결정

# 입력층 및 출력층
all_weights[0] = np.random.normal(size=(D, D_i))  # 입력층
all_weights[-1] = np.random.normal(size=(D_o, D))  # 출력층
all_biases[0] = np.random.normal(size =(D,1))  # 입력층
all_biases[-1]= np.random.normal(size =(D_o,1))  # 출력층

# 중간 층 생성
for layer in range(1,K):
  all_weights[layer] = np.random.normal(size=(D,D))
  all_biases[layer] = np.random.normal(size=(D,1))

**배열(Array)** <br>
배열의 크기는 고정되어 있으며, 선언 시에 크기를 지정해야함 <br><br>
**리스트(List)** <br>
원소의 개수가 가변적이며, 삽입과 삭제가 자유롭다. <br><br>

->  다양한 신경망 구조를 다루기 위해서는 각 신경망에 따라 필요한 파라미터의 크기가 다르기 때문에 <br>
      리스트를 사용하여 미리 공간을 할당하고, 필요한 시점에 배열을 생성하여      해당 위치에 저장

\



---



### 2. Forward pass

- ReLU Function 정의

In [ ]:
# ReLU function 정의
def ReLU(preactivation):
  activation = preactivation.clip(0.0)
  return activation

- 신경망의 순방향(forward pass) 계산

In [ ]:
def compute_network_output(net_input, all_weights, all_biases):

  K = len(all_weights) -1   # 5

  # all_f: 각 층의 사전 활성화 함수
  # all_h: 각 층의 활성화 함수
  all_f = [None] * (K+1)
  all_h = [None] * (K+1)

  # 입력층
  all_h[0] = net_input

  # 중간층
  for layer in range(K):  # 0~4
      all_f[layer] = all_biases[layer] + np.matmul(all_weights[layer], all_h[layer])
      all_h[layer+1] = ReLU(all_f[layer])

  # 출력층
  all_f[K] = all_biases[K] + np.matmul(all_weights[K], all_h[K])
  net_output = all_f[K]

  return net_output, all_f, all_h

In [ ]:
# check

net_input = np.ones((D_i,1)) * 1.2
net_output, all_f, all_h = compute_network_output(net_input,all_weights, all_biases)
print("True output = %3.3f, Your answer = %3.3f"%(1.907, net_output[0,0]))

True output = 1.907, Your answer = 1.907


\



---



### 3. Backward pass

- 손실 함수 정의 및 출력에 대한 손실함수의 도함수 계산

In [ ]:
def least_squares_loss(net_output, y):
  return np.sum((net_output-y) * (net_output-y))

def d_loss_d_output(net_output, y):
    return 2*(net_output -y);

In [ ]:
y = np.ones((D_o,1)) * 20.0
loss = least_squares_loss(net_output, y)
print("y = %3.3f Loss = %3.3f"%(y, loss))

y = 20.000 Loss = 327.371


<ipython-input-20-811c7c3c79c2>:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("y = %3.3f Loss = %3.3f"%(y, loss))


- 신경망의 backward pass

In [ ]:
# indicator function
def indicator_function(x):
  x_in = np.array(x)
  x_in[x_in>=0] = 1
  x_in[x_in<0] = 0
  return x_in

def backward_pass(all_weights, all_biases, all_f, all_h, y):
  # 편향, 가중치, f, h에 대한 손실함수의 도함수 저장
  all_dl_dweights = [None] * (K+1)
  all_dl_dbiases = [None] * (K+1)
  all_dl_df = [None] * (K+1)
  all_dl_dh = [None] * (K+1)

  # 출력에 대한 손실의 도함수 계산
  all_dl_df[K] = np.array(d_loss_d_output(all_f[K],y))

  # 역방향으로 작동
  for layer in range(K,-1,-1):  # k(5)~0 (역방향으로 작동)
    all_dl_dbiases[layer] = all_dl_df[layer]
    all_dl_dweights[layer] = np.matmul(all_dl_df[layer], all_h[layer].T)
    all_dl_dh[layer] = np.matmul(all_weights[layer].T, all_dl_df[layer])

    if layer > 0:
      all_dl_df[layer-1] = all_dl_dh[layer] * indicator_function(all_f[layer-1])

  return all_dl_dweights, all_dl_dbiases

In [ ]:
all_dl_dweights, all_dl_dbiases = backward_pass(all_weights, all_biases, all_f, all_h, y)

\



---



### 4. 유한차분(finite difference)을 사용한 역전파 구현의 정확성 검증

<br>
$
f'(x) ≈  \frac{f(x+ϵ) - f(x)}{ϵ}
$
<br><br>

In [ ]:
# 출력 설정
np.set_printoptions(precision=3)  # 출력 시 소수점은 셋째자리 까지만 출력함.

# 유한차분법으로 계산된 파라미터 값 저장
all_dl_dweights_fd = [None] * (K+1)
all_dl_dbiases_fd = [None] * (K+1)

# 미세변화량 설정
delta_fd = 0.000001

# 편향의 기울기 검정
for layer in range(K):

  # 현재 층의 편향에 대한 손실함수의 도함수 저장 공간 설정
  dl_dbias  = np.zeros_like(all_dl_dbiases[layer]) # 현재 층의 바이어스의 기울기 배열

  for row in range(all_biases[layer].shape[0]): # 현재 층의 바이어스의 행 개수(길이) ex) all_dbiases[0].shape[0] -> 6

    # 편향의 변화
    all_biases_copy = [np.array(x) for x in all_biases] # 모든 바이어스 배열의 복사본 -> 기존배열을 변경하지 않기 위해
    all_biases_copy[layer][row] += delta_fd

    # 신경망 계산
    network_output_1, *_ = compute_network_output(net_input, all_weights, all_biases_copy)
    network_output_2, *_ = compute_network_output(net_input, all_weights, all_biases)

    # 유한차분법을 이용한 도함수 계산
    dl_dbias[row] = (least_squares_loss(network_output_1, y) - least_squares_loss(network_output_2,y))/delta_fd

  all_dl_dbiases_fd[layer] = np.array(dl_dbias)
  print("-----------------------------------------------")
  print("Bias %d, derivatives from backprop:"%(layer))
  print(all_dl_dbiases[layer])
  print("Bias %d, derivatives from finite differences"%(layer))
  print(all_dl_dbiases_fd[layer])

  if np.allclose(all_dl_dbiases_fd[layer],all_dl_dbiases[layer],rtol=1e-05, atol=1e-08, equal_nan=False):
    # equal_nan:  NaN 값이 있는 경우에도 두 배열이 완전히 같아야 함.
    print("Success!  Derivatives match.")
  else:
    print("Failure!  Derivatives different.")



# 가중치의 기울기 검정
for layer in range(K):
  dl_dweight  = np.zeros_like(all_dl_dweights[layer])
  # For every element in the bias
  for row in range(all_weights[layer].shape[0]):
    for col in range(all_weights[layer].shape[1]):
      # Take copy of biases  We'll change one element each time
      all_weights_copy = [np.array(x) for x in all_weights]
      all_weights_copy[layer][row][col] += delta_fd
      network_output_1, *_ = compute_network_output(net_input, all_weights_copy, all_biases)
      network_output_2, *_ = compute_network_output(net_input, all_weights, all_biases)
      dl_dweight[row][col] = (least_squares_loss(network_output_1, y) - least_squares_loss(network_output_2,y))/delta_fd
  all_dl_dweights_fd[layer] = np.array(dl_dweight)
  print("-----------------------------------------------")
  print("Weight %d, derivatives from backprop:"%(layer))
  print(all_dl_dweights[layer])
  print("Weight %d, derivatives from finite differences"%(layer))
  print(all_dl_dweights_fd[layer])
  if np.allclose(all_dl_dweights_fd[layer],all_dl_dweights[layer],rtol=1e-05, atol=1e-08, equal_nan=False):
    print("Success!  Derivatives match.")
  else:
    print("Failure!  Derivatives different.")

-----------------------------------------------
Bias 0, derivatives from backprop:
[[ -4.486]
 [  4.947]
 [  6.812]
 [ -3.883]
 [-24.935]
 [  0.   ]]
Bias 0, derivatives from finite differences
[[ -4.486]
 [  4.947]
 [  6.812]
 [ -3.883]
 [-24.935]
 [  0.   ]]
Success!  Derivatives match.
-----------------------------------------------
Bias 1, derivatives from backprop:
[[ -0.   ]
 [-11.297]
 [  0.   ]
 [  0.   ]
 [-10.722]
 [  0.   ]]
Bias 1, derivatives from finite differences
[[  0.   ]
 [-11.297]
 [  0.   ]
 [  0.   ]
 [-10.722]
 [  0.   ]]
Success!  Derivatives match.
-----------------------------------------------
Bias 2, derivatives from backprop:
[[-0.   ]
 [-0.   ]
 [ 0.938]
 [ 0.   ]
 [-9.993]
 [ 0.508]]
Bias 2, derivatives from finite differences
[[ 0.   ]
 [ 0.   ]
 [ 0.938]
 [ 0.   ]
 [-9.993]
 [ 0.508]]
Success!  Derivatives match.
-----------------------------------------------
Bias 3, derivatives from backprop:
[[-0.   ]
 [-4.8  ]
 [-1.661]
 [-0.   ]
 [ 3.393]
 [ 5.391]


```
np.allclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)
```

> a : 첫 번째 입력 배열. <br>
b : 두 번째 입력 배열.<br>
rtol : 상대 허용 오차 (relative tolerance). 기본값은 1e-05.<br>
atol : 절대 허용 오차 (absolute tolerance). 기본값은 1e-08.<br>
equal_nan : True로 설정하면 NaN 값들도 서로 같다고 간주함. 기본값은 False.


 두 배열의 요소들이 모두 주어진 허용 오차 범위 내에서 가까운지 여부를 확인하는 데 사용하는 함수이며, 두 배열의 모든 요소가 허용 오차 내에서 가까우면 True, 그렇지 않으면 False를 반환함.

   